In [1]:
import math
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [2]:
shape = (224,224,3)
numclass = 1000

In [3]:
def shortcut(x,dwkernel,strides,out):
    x = layers.DepthwiseConv2D(dwkernel,strides,padding='same',use_bias=False)(x)
    x = layers.BatchNormalization(axis=-1)(x)
    x = layers.Conv2D(out,1,1,padding='same',use_bias=False)(x)
    x = layers.BatchNormalization(axis=-1)(x)
    return x

In [4]:
def SEBlock(inputs,se_ratio = 4):
    filters = int(inputs.shape[3])
    reduced_filters  = int(filters/se_ratio)
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.Reshape((1,1,x.shape[1]))(x)
    x = layers.Conv2D(reduced_filters, 1, 1,use_bias=True)(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 1, 1,use_bias=True)(x)
    x = layers.Activation('hard_sigmoid')(x)
    x = layers.Multiply()([x, inputs])
    return x

def GhostModule(x,outchannels,ratio,convkernel,dwkernel,strides=1,relu = True):
    init_channels = math.ceil(outchannels / ratio)
    x = layers.Conv2D(int(init_channels),convkernel,strides,padding='same',use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    if relu:
        x = layers.ReLU()(x)
    
    x1 = layers.DepthwiseConv2D(dwkernel,strides,padding='same',depth_multiplier=ratio-1,use_bias=False)(x)
    x1 = layers.BatchNormalization()(x1)
    if relu:
        x1 = layers.ReLU()(x1)
    
    out = layers.Concatenate(axis=-1)([x,x1])
    return out[:,:,:,:outchannels]


def GhostBottleneck(x,dwkernel,strides,exp,out,ratio,use_se):
    y = GhostModule(x,exp,ratio,1,3,relu=True)
    if(strides>1):
        y = layers.DepthwiseConv2D(dwkernel,strides,padding='same',use_bias=False)(y)
        y = layers.BatchNormalization()(y)
    if(use_se==True):
        y = SEBlock(y)
    y = GhostModule(y,out,ratio,1,3,relu=False)
    if int(x.shape[3])!= int(y.shape[3]):
        x = shortcut(x,dwkernel,strides,out)
    y = layers.add([x,y])
    return y

In [5]:
def GhostNet():
    inputdata = layers.Input(shape)
        
    x = layers.Conv2D(16,3,2,padding='same',use_bias=False)(inputdata)
    x = layers.BatchNormalization(axis=-1)(x)
    x = layers.Activation('relu')(x)

    x = GhostBottleneck(x,3,1,16,16,2,False)
    x = GhostBottleneck(x,3,2,48,24,2,False)
    x = GhostBottleneck(x,3,1,72,24,2,False)
    x = GhostBottleneck(x,5,2,72,40,2,True)
    x = GhostBottleneck(x,5,1,120,40,2,True)
    x = GhostBottleneck(x,3,2,240,80,2,False)
    x = GhostBottleneck(x,3,1,200,80,2,False)
    x = GhostBottleneck(x,3,1,184,80,2,False)
    x = GhostBottleneck(x,3,1,184,80,2,False)
    x = GhostBottleneck(x,3,1,480,112,2,True)
    x = GhostBottleneck(x,3,1,672,112,2,True)
    x = GhostBottleneck(x,5,2,672,160,2,True)
    x = GhostBottleneck(x,5,1,960,160,2,False)
    x = GhostBottleneck(x,5,1,960,160,2,True)
    x = GhostBottleneck(x,5,1,960,160,2,False)
    x = GhostBottleneck(x,5,1,960,160,2,True)
    
    x = layers.Conv2D(960,1,1,padding='same',use_bias=False)(x)
    x = layers.BatchNormalization(axis=-1)(x)
    x = layers.Activation('relu')(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Reshape((1,1,int(x.shape[1])))(x)
    x = layers.Conv2D(1280,1,1,padding='same',use_bias=True)(x)
    x = layers.BatchNormalization(axis=-1)(x)
    x = layers.Activation('relu')(x)

    x = layers.Dropout(0.05)(x)
    x = layers.Conv2D(numclass,1,1,padding='same',use_bias=False)(x)
    x = K.squeeze(x,1)
    x = K.squeeze(x,1)
    out = keras.activations.softmax(x)

    model = keras.Model(inputdata, out)
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),metrics=['accuracy']) 
    return model            

In [6]:
x = GhostNet()

In [7]:
x.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________